In [75]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import os
import sys

In [7]:
def get_response(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0"
    }
    return requests.get(url, headers=headers)

In [63]:
def get_book_data_by_isbn(target_isbn_code):
    full_book_info = get_response("https://www.googleapis.com/books/v1/volumes?q=isbn:" + target_isbn_code)
    print("Book Data Before Processing...")
    full_book_info_json = full_book_info.json()
    print(full_book_info_json)
    book_volumeInfo = full_book_info_json.get('items')[0].get("volumeInfo")
    return {
        "isbn": target_isbn_code,
        "title": book_volumeInfo.get("title"),
        "authors": book_volumeInfo.get("authors"),
        "description": book_volumeInfo.get("description"),
        "categories": book_volumeInfo.get("categories"),
    }


In [41]:
# get recommend book from https://www.shoeisha.co.jp/campaign/award/vote
recommend_response = get_response("https://www.shoeisha.co.jp/campaign/award/vote")
recommend_soup = BeautifulSoup(recommend_response.text)
all_divs = recommend_soup.find_all('div', class_='col-md-2 col-6 mb-3 isbn')
all_isbn = [div.attrs['rel'] for div in all_divs if 'rel' in div.attrs]
len(all_isbn)

62

In [64]:
# save book data to json file
results = []
for isbn in all_isbn:
    book_data = get_book_data_by_isbn(isbn)
    results.append(book_data)

with open('recommend_book_data.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

Book Data Before Processing...
{'kind': 'books#volumes', 'totalItems': 1, 'items': [{'kind': 'books#volume', 'id': 'r5HPEAAAQBAJ', 'etag': 'ndxuPNvZdrA', 'selfLink': 'https://www.googleapis.com/books/v1/volumes/r5HPEAAAQBAJ', 'volumeInfo': {'title': 'アジャイルプラクティスガイドブック チームで成果を出すための開発技術の実践知', 'authors': ['常松 祐一'], 'publisher': '翔泳社', 'publishedDate': '2023-07-20', 'description': 'チーム・組織にプラクティスを導入し、根付かせるために！ 116の手法を一冊にまとめた“実践”の手引き チームでのアジャイル開発には、開発技術やツールなどの「技術プラクティス」の活用が重要です。 プラクティスはそれぞれの目的や役割を意識することで効果を発揮します。しかし、目まぐるしく状況が変化する開発では、当初の目的を忘れて、プラクティスに取り組むこと自体が目的化してしまうチームも少なくありません。 本書は、チーム・組織でアジャイル開発に取り組んできた著者が、プラクティスの効果的な選択・活用のしかたについて、自らの実践経験に基づいてまとめたガイドブックです。 架空の開発現場を舞台にしたマンガとともに、チーム開発の様々なシーンで役立てられるプラクティスを、幅広くかつわかりやすく解説しています。開発現場に備えておけば、特定のプラクティスについて知りたい、開発の段階に合わせたプラクティスを探したい、といった場面で、必要な項目を調べる辞書として役立てることができるでしょう。プラクティスの導入や実践について、試行錯誤を重ねている開発者におすすめの一冊です。 ●本書で取り上げる開発のシーン 実装方針の検討、タスクの分解、ブランチ戦略の検討、コミット、コードレビュー、複数人での共同作業、テスト、運用を見据えたソースコードの整備、CI/CD、デプロイ、リリース、モニタリング、関係者間の認識合わせ、チーム内外との連携 …など、数々の場面で

In [65]:
# get other books' isbn code
java_book = "スッキリわかるJava入門,やさしいJava 第7版 (「やさしい」シリーズ),スラスラ読める Javaふりがなプログラミング ふりがなプログラミングシリーズ,新・明解Java入門 第2版 (新・明解シリーズ),ゼロからわかる Java超入門 [改訂3版],いちばんやさしいJavaの教本 人気講師が教えるプログラミングの基礎,プロになるJava―仕事で必要なプログラミングの知識がゼロから身につく最高の指南書,Java 第3版 実践編 アプリケーション作りの基本,新わかりやすいJava オブジェクト指向徹底解説 第2版,スッキリわかるJava入門 実践編 第3版"

cpp_book = "スラスラわかるC++,新・明解C++入門,新・明解C++で学ぶオブジェクト指向プログラミング,現場ですぐに使える！ C/C++逆引き大全 560の極意,C++ポケットリファレンス,基礎からしっかり学ぶC++の教科書 C++14対応,プログラミング言語C++"

vue_book = "Vue.jsビギナーズガイド 3.x対応,現場で使えるVue.js 3.x実践ガイド,Vue 3 フロントエンド開発の教科書,動かして学ぶ！Vue.js開発入門,改訂新版　Vue.jsとFirebaseで作るミニWebサービス,現場で使えるVue.js tips集"

springboot_book = "はじめてのSpring Boot,後悔しないためのSpring Boot入門書,現場至上主義 Spring Boot2 徹底活用"

psql_book = "データベース初心者のためのPostgreSQL教室,PostgreSQL徹底入門 第4版,内部構造から学ぶPostgreSQL 設計・運用計画の鉄則,SQLデータ分析・活用入門 データサイエンスの扉を開くための技術 MySQL/PostgreSQL 両対応,SQL実践入門 ──高速でわかりやすいクエリの書き方"

aws_book = "ゼロからわかるamazon web services超入門 はじめてのクラウド,図解即戦力 amazon web servicesのしくみと技術がこれ1冊でしっかりわかる教科書,AWSクラウドの基本と仕組み ,Amazon Web Services 基礎からのネットワーク&サーバー構築 改訂3版,Amazon Web Services 業務システム設計・移行ガイド (Informatics&IDEA),Amazon Web Services ネットワーク入門 (impress top gear)"

ts_book = "ゼロからわかる　TypeScript入門,プロを目指す人のためのTypeScript入門　安全なコードの書き方から高度な型の使い方まで Software Design plus,プログラミングTypeScript ―スケールするJavaScriptアプリケーション開発,手を動かしながら学ぶ TypeScript,TypeScriptとReact/Next.jsでつくる実践Webアプリケーション開発"

In [66]:
java_book_list = java_book.split(",")
cpp_book_list = cpp_book.split(",")
springboot_book_list = springboot_book.split(",")
vue_book_list = vue_book.split(",")
psql_book_list = psql_book.split(",")
aws_book_list = aws_book.split(",")
ts_book_list = ts_book.split(",")

In [100]:
def get_book_data_by_title(target_book_name):
    try:
        full_book_info = get_response("https://www.googleapis.com/books/v1/volumes?q=title:" + target_book_name)
        # print("Book Data Before Processing...")
        full_book_info_json = full_book_info.json()
        # print(full_book_info_json)
        book_volumeInfo = full_book_info_json.get('items')[0].get("volumeInfo")
        return {
            "isbn": [_ for _ in book_volumeInfo.get("industryIdentifiers") if _["type"] == "ISBN_13"][0].get("identifier"),
            "title": book_volumeInfo.get("title"),
            "authors": book_volumeInfo.get("authors"),
            "description": book_volumeInfo.get("description"),
            "categories": book_volumeInfo.get("categories"),
        }
    except Exception as e:
        print("Error", e)

In [104]:
def save_data(book_list, save_name):
    results = []
    for book in tqdm(book_list):
        book_data = get_book_data_by_title(book)
        results.append(book_data)
    with open(os.path.join("data", save_name+'.json'), 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

In [105]:
save_data(java_book_list, "java_book_data")
save_data(cpp_book_list, "cpp_book_data")
save_data(springboot_book_list, "springboot_book_data")
save_data(vue_book_list, "vue_book_data")
save_data(psql_book_list, "psql_book_data")
save_data(aws_book_list, "aws_bookdata")
save_data(ts_book_list, "ts_book_data")

 90%|█████████ | 9/10 [00:08<00:00,  1.01it/s]

Error list index out of range


 67%|██████▋   | 2/3 [00:01<00:00,  1.22it/s]

Error list index out of range


100%|██████████| 6/6 [00:04<00:00,  1.30it/s]


Error 'NoneType' object is not iterable


 20%|██        | 1/5 [00:00<00:03,  1.11it/s]

Error 'NoneType' object is not iterable


100%|██████████| 5/5 [00:04<00:00,  1.19it/s]


In [107]:
# combine all data
data_list = os.listdir("data")
books_list = []
for data in tqdm(data_list):
    data_file = os.path.join("data", data)
    with open(data_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        books_list.extend([_ for _ in data])

with open('combined_book_data.json', 'w', encoding='utf-8') as f:
    json.dump(books_list, f, ensure_ascii=False, indent=4)

100%|██████████| 8/8 [00:00<00:00, 7998.67it/s]
